# load libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import re
import glob
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



# crawl the website to get the data

## 1. Kimchi Lab

In [ ]:
//*[@id="contents_body"]/div[2]/div[3]/div[2]/div[1]/ul/li/strong
//*[@id="contents_body"]/div[2]/div[3]/div[2]/div[1]/ul/li/span

//*[@id="contents_body"]/div[2]/div[3]/div[2]/div[2]/ul/li[1]/strong
//*[@id="contents_body"]/div[2]/div[3]/div[2]/div[2]/ul/li/span

In [105]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

# "검색어: N번째" 목록을 dict로 구성
search_dict = {
    '백김치': [5],
    '갓김치': [3],
    '동치미': [3],
    '고들빼기': [2],    # 1
    '배추김치': [4],
    '열무김치': [2],
    '얼갈이': [2],
    '총각': [2],
    '오이': [2],
    '파김치': [5],
    '부추김치': [1]
}

wait = WebDriverWait(driver, 10)
results = []

for keyword, nth_list in search_dict.items():
    print(f"\n=== [검색어: {keyword}] ===")
    
    # (1) 검색 페이지로 새로 진입
    driver.get('https://www.wikim.re.kr/archiveList.es?mid=a50301040100&srh_ai_code=DATA002')
    
    # 검색창 로딩 대기
    search_box = wait.until(
        EC.presence_of_element_located(
            (By.XPATH, '//*[@id="srhForm"]/fieldset/div/span[1]/input')
        )
    )
    # 검색창 초기화 후 검색어 입력
    search_box.clear()
    search_box.send_keys(keyword)

    # 검색 버튼 클릭
    search_btn = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="srhForm"]/fieldset/div/span[2]/span/span/button')
        )
    )
    driver.execute_script("arguments[0].click();", search_btn)
    
    time.sleep(1)

    for nth in nth_list:
        print(f"  - {keyword} {nth}번째 레시피 확인...")
        try:
            # n번째 아이템 XPATH
            li_xpath = f'/html/body/div[5]/main/div/section[2]/div[2]/div[3]/ul/li[{nth}]'
            li_element = wait.until(EC.presence_of_element_located((By.XPATH, li_xpath)))
            
            # 상세페이지로 이동
            a_tag = li_element.find_element(By.TAG_NAME, 'a')
            driver.execute_script("arguments[0].click();", a_tag)

            time.sleep(2)

            # === 여기서부터는 "제공해주신 코드" 그대로 ===
            ### ① 제목
            title = driver.find_element(By.XPATH, '//*[@id="contents_body"]/div[2]/div[1]/div/strong').text
            print(f"    📌 제목: {title}")

            ### ② 태그 목록 (개수 유동적)
            tags = driver.find_elements(By.XPATH, '//*[@id="contents_body"]/div[2]/div[1]/div/div[1]/a/span')
            tag_list = [tag.text for tag in tags]
            print(f"    🏷 태그: {', '.join(tag_list)}")

            ### ③ 김치 설명
            try:
                info_element = driver.find_element(By.XPATH, '//*[@id="contents_body"]/div[2]/div[1]/div/div[2]')
                info = info_element.text.strip()
            
                if not info:
                    # 보조 설명 위치에서 재시도
                    alt_info_element = driver.find_element(By.XPATH, '//*[@id="contents_body"]/div[2]/div[2]/div')
                    info = alt_info_element.text.strip()
            except:
                info = ""  # 예외 발생 시 빈 문자열 처리
            
            print(f"    📝 김치 설명:\n{info}")

            ### ④ 재료 목록 (다중 XPath fallback)
            print("    🥬 재료 목록:")
            ing_list = []
            
            # 재료 XPath 목록 (우선순위 순서)
            ingredient_xpaths = [
                '//*[@id="contents_body"]/div[2]/div[2]/div[2]/div/ul/li',
                '//*[@id="contents_body"]/div[2]/div[3]/div[2]/div[1]/ul/li',
                '//*[@id="contents_body"]/div[2]/div[2]/div/div/ul/li'
            ]
            
            found = False
            for xpath in ingredient_xpaths:
                try:
                    ingredients = driver.find_elements(By.XPATH, xpath)
                    if ingredients:
                        for ing in ingredients:
                            try:
                                name_el = ing.find_element(By.TAG_NAME, 'strong')
                                amount_el = ing.find_element(By.TAG_NAME, 'span')
                                ing_list.append((name_el.text.strip(), amount_el.text.strip()))
                                print(f"      - {name_el.text.strip()}: {amount_el.text.strip()}")
                            except:
                                # 태그 누락 시 strong만 있는 경우도 처리
                                try:
                                    name = ing.find_element(By.TAG_NAME, 'strong').text.strip()
                                    ing_list.append((name, ""))
                                    print(f"      - {name}")
                                except:
                                    continue
                        found = True
                        break  # 첫 번째로 성공한 xpath에서만 처리
                except Exception as e:
                    continue
            
            if not found:
                print("      [재료 없음 또는 XPath 불일치]")
            
            ### ⑤ 만드는 방법 텍스트 (보조 XPath 포함)
            print("    🍳 만드는 방법:")
            steps_list = []
            try:
                steps_text = driver.find_elements(By.XPATH, '//*[@id="contents_body"]/div[2]/div[3]/ol/li/div[1]')
                if not steps_text:
                    steps_text = driver.find_elements(By.XPATH, '//*[@id="contents_body"]/div[2]/div[4]/ol/li/div[1]')
                
                for i, step in enumerate(steps_text, 1):
                    step_text = step.text.strip()
                    steps_list.append(step_text)
                    print(f"      {i}. {step_text}")
            except Exception as e:
                print(f"      [조리 텍스트 오류]: {e}")
            
            ### ⑥ 만드는 방법 이미지 (보조 XPath 포함)
            print("    🖼 조리 이미지:")
            img_list = []
            try:
                steps_img = driver.find_elements(By.XPATH, '//*[@id="contents_body"]/div[2]/div[3]/ol/li/div[2]/img')
                if not steps_img:
                    steps_img = driver.find_elements(By.XPATH, '//*[@id="contents_body"]/div[2]/div[4]/ol/li/div[2]/img')
                
                for i, img in enumerate(steps_img, 1):
                    img_src = img.get_attribute('src')
                    img_list.append(img_src)
                    print(f"      {i}. {img_src}")
            except Exception as e:
                print(f"      [조리 이미지 오류]: {e}")

            # 결과 저장
            data = {
                "keyword": keyword,
                "nth": nth,
                "title": title,
                "tags": tag_list,
                "info": info,
                "ingredients": ing_list,
                "steps_text": steps_list,
                "steps_img": img_list
            }
            results.append(data)

            # 상세페이지 끝났으면, 브라우저 뒤로가기 OR 그냥 다음 nth로 이동
            driver.back()
            time.sleep(1)

        except Exception as e:
            print(f"    [오류] {nth}번째 접근 중 예외 발생: {e}")

# 전체 완료
print("\n=== 크롤링 완료 ===")
print(f"총 {len(results)}건의 레시피 정보를 수집했습니다.")
# driver.quit()



=== [검색어: 백김치] ===
  - 백김치 5번째 레시피 확인...
    📌 제목: 백김치
    🏷 태그: #발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #김치, #백김치
    📝 김치 설명:
국물이 자박자박하도록 김치를 담구는 백김치는 예전에는 김치 국물로 사골 국물을 넣기도 했으며 단맛을 내기 위해 과일을 사용한다.
    🥬 재료 목록:
      - 배추: 15kg
    🍳 만드는 방법:
      1. 1. 배추는 시든 겉잎을 떼고 작은 것은 반으로, 큰 것은 네 개로 가른다.

2. 굵은 소금을 조금 남겨두고 물에 풀어 소금물을 만든다.

배추를 소금물에 담갔다가 건져서, 배추 줄기 쪽에 소금을 조금씩 뿌려 큰 통에 담아 절인다.

절이는 도중에 위아래를 바꾸어 전체가 고루 절게 한다.
      2. 3. 절인 배추는 찬물에 헹구어 채반에 받쳐 물기를 뺀다. 지저분한 뿌리 부분은 잘라낸다.

4. 무는 깨끗이 씻어서 물기를 뺀 후 2mm 폭으로 채 썬다.(1/2개는 남겨둔다.)
      3. 5. 쪽파, 갓, 미나리, 대파는 깨끗이 손질한 후에 잘게 썬다.
      4. 6. 생강과 마늘은 곱게 채 썬다.

7. 새우젓은 건더기만 건져 굵게 다지고 젓국은 남긴다.

8. 밤은 채 썰고 대추는 씨를 발라내어 채 썬다.

9. 배는 하나만 껍질을 벗겨서 채 썰고 나머지는 네 조각으로 쪼갠다.

10. 석이버섯은 따뜻한 물에 불려서 지저분한 것을 떼어내고 돌돌 말아 채 썬다.

11. 표고버섯은 따뜻한 물에 불려서 채 썬다.

12. 실고추는 3cm 길이로 자른다.
      5. 13. 넓은 그릇에 채 썬 무, 배, 밤, 대추를 담고 실고추를 넣고 잘 섞는다.

14. 미나리, 갓, 쪽파와 채 썬 파, 마늘, 생강, 석이, 표고 등의 소 재료를 넣어 살살 버무린다.

15. 곱게 다진 새우젓과 소금, 설탕을 넣어 간을 맞추고 맛을 내어 소를 만든다.
      6. 16. 준비한 소를 절인 배춧잎의 사이사이에 고루

In [107]:
data = {
    "keyword": keyword,
    "nth": nth,
    "title": title,
    "tags": tag_list,           # ex) ["#발효음식", "#배추김치"]
    "info": info,               # ex) "김치는 한국 전통의..."
    "ingredients": ing_list,    # ex) [("배추", "2포기"), ("소금", "1컵")]
    "steps_text": steps_list,   # ex) ["먼저 배추를 절인다", "양념을 만든다", ...]
    "steps_img": img_list       # ex) ["https://...", "https://...", ...]
}
import pandas as pd

# results 라는 리스트 안에 위 형태의 딕셔너리들이 여러 개 들어있다고 가정
df = pd.DataFrame(results)


In [109]:
df

,keyword,nth,title,tags,info,ingredients,steps_text,steps_img
0,백김치,5,백김치,"[#발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #김치, #백김치]",국물이 자박자박하도록 김치를 담구는 백김치는 예전에는 김치 국물로 사골 국물을 넣기...,"[(배추, 15kg)]","[1. 배추는 시든 겉잎을 떼고 작은 것은 반으로, 큰 것은 네 개로 가른다.\n\...",[https://www.wikim.re.kr/archiveImgView.es?mid...
1,갓김치,3,갓김치,"[#발효음식, #종가, #종부, #내림음식, #진안, #괴정고택, #김장, #김치,...",,"[(청갓, 3단), (당근, 1개), (쪽파, 400g), (고들빼기, 400g),...","[1. 고들빼기는 시든 잎과 잔털을 제거하여 물에 담가 4일간 삭힌다., 2. 청갓...",[https://www.wikim.re.kr/archiveImgView.es?mid...
2,동치미,3,동치미,"[#발효음식, #종가, #종부, #내림음식, #대전, #은진송씨송준길종가, #동춘당...",,"[(무, 5개), (소금, 20g), (삭힌 고추, 20개), (파, 200g), ...",[1. 고추를 소금물에 담가 무거운 것으로 눌러 고추가 누렇게 변할 때 까지 삭힌다...,[https://www.wikim.re.kr/archiveImgView.es?mid...
3,고들빼기,2,고들빼기김치,"[#종가김치, #이애섭 종녀, #고들빼기, #고들빼기김치]",전라도 지역에서 많이 담그는 김치로 겨울 김장김치 중 하나이다. 고들빼기를 소금물에...,"[(고들빼기, 1.4kg), (소금, 500g), (굵은 고춧가루, 200g), (...",[1. 고들빼기는 칼끝으로 뿌리를 다듬고 씻은 후 소금물에 절인다. 절여진 고들빼기...,[https://www.wikim.re.kr/archiveImgView.es?mid...
4,배추김치,4,배추김치,"[#발효음식, #종가, #종부, #내림음식, #대전, #은진 송씨 송준길 종가, #...",대전 은진 송씨 송준길 종가의 내림음식으로 종가 사람들이 국물이 있는 김치를 선호하...,"[(절임 배추, 2포기), (무(섞박지 모양), 800g), (무(채썬 것), 70...","[1. 배추는 푹 절이지 않고 살짝 덜 절여진 것을 쓴다., 2. 무의 일부는 0....",[https://www.wikim.re.kr/archiveImgView.es?mid...
5,열무김치,2,열무김치,"[#발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #김치, #열무...",여름김치에는 젓갈을 사용하지 않는다.\n\n물김치로 만들 때는 국물이 없을 때 보다...,"[(열무, 5kg)]","[1. 열무는 억세고 시든 잎은 떼어내고, 뿌리는 끝쪽만 손질한다., 2. 열무에 ...",[https://www.wikim.re.kr/archiveImgView.es?mid...
6,얼갈이,2,얼갈이김치,"[#발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #김치, #얼갈...",열무김치와 재료가 거의 동일하고 국물이 자박자박하게 김칫국물을 붓는다.,"[(얼갈이 배추, 1.5kg)]","[1. 얼갈이배추는 먹기 좋은 길이로 잘라 여러 번 씻는다., 2. 소금물에 얼갈이...",[https://www.wikim.re.kr/archiveImgView.es?mid...
7,총각,2,총각김치,"[#발효음식, #종가, #종부, #내림음식, #진안, #괴정고택, #총각무, #알타...",,"[(총각무, 1단), (소금, 200g), (물, 2L), (쪽파, 100g), (...",[1. 총각무는 무와 무청 사이사이를 칼로 깨끗하게 다듬는다.\n\n2. 물에 소금...,[https://www.wikim.re.kr/archiveImgView.es?mid...
8,오이,2,오이통지,"[#발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #오이통지, #...",● 오이는 조선오이(백오이)를 사용한다.\n● 모든 재료는 국물을 떠 먹을 때 편하...,"[(오이(백오이), 1625g)]","[1. 오이는 양 끝을 잘라내고 4등분하여 열십자로 칼집을 낸다., 2. 물에 소금...",[https://www.wikim.re.kr/archiveImgView.es?mid...
9,파김치,5,파김치,"[#발효음식, #종가, #종부, #내림음식, #의정부, #서계고택, #김치, #파,...","서울, 경기 지역의 경우 김치를 담글 때 젓갈을 많이 쓰지 않는다.\n젓갈을 사용할...","[(골파, 2kg), (물, 1컵), (찹쌀가루, 2큰술), (고춧가루, 160g)...",[1. 냄비에 물과 찹쌀가루를 넣고 끓여 찹쌀풀을 만든다.\n\n2. 골파는 뿌리를...,[https://www.wikim.re.kr/archiveImgView.es?mid...


In [68]:
df.iloc[0]['steps_img']

['https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=306',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=307',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=308',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=309',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=310',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=311',
 'https://www.wikim.re.kr/archiveImgView.es?mid=a50301040100&ai_seq=81&af_seq=312']

## 2. Temple Food

In [ ]:
#contents > div > div.recipe_list > div > div.recipe_how_slick > div > div > div > div:nth-child(1) > p

In [ ]:
#contents > div > div.recipe_list > div > div.recipe_how_slick > div > div > div > div:nth-child(2) > p

In [ ]:
//*[@id="contents"]/div/div[1]/div/div[3]/div/div/div/div[2]/p
//*[@id="contents"]/div/div[1]/div/div[3]/div/div/div/div[3]/p

In [103]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

base_url = "https://www.koreatemplefood.com/templefood/recipe/index.html?page={}&plist=&find_field=title&find_word=&find_state=&find_ordby=&conf=&mode=&jijum_uid=&s_season22=&s_food22=%EA%B9%80%EC%B9%98%EB%A5%98"

results = []

try:
    # 1) 페이지 루프
    for page in range(1, 14):
        print(f"\n=== [템플푸드] 페이지 {page} ===")
        driver.get(base_url.format(page))
        time.sleep(1)
        
        # 각 페이지에 6개의 레시피 목록
        for i in range(1, 7):
            print(f"  - {page}페이지, {i}번째 레시피 확인 중...")
            try:
                # (1) 레시피 이름
                name_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[2]'
                name_element = wait.until(EC.presence_of_element_located((By.XPATH, name_xpath)))
                name = name_element.text.strip()
                
                # (2) 썸네일 (CSS background-image)
                thumb_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[1]/span'
                thumb_element = driver.find_element(By.XPATH, thumb_xpath)
                # background-image: url("...")
                thumb_style = thumb_element.value_of_css_property('background-image')
                if 'url("' in thumb_style:
                    thumbnail_url = thumb_style.split('url("')[1].split('")')[0]
                else:
                    thumbnail_url = ""
                
                # (3) 태그 목록 (여러 개)
                tag_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[1]/span'
                tag_elements = driver.find_elements(By.XPATH, tag_xpath)
                tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

                # (4) 상세페이지 진입
                list_item = driver.find_element(By.XPATH, f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a')
                driver.execute_script("arguments[0].click();", list_item)
                time.sleep(1)
                
                # (5) 재료 수집
                print("    🥬 재료 목록:")
                ingredients = []
                idx_ing = 1
                while True:
                    ing_xpath = f'//*[@id="contents"]/div/div[1]/div/ol/li[{idx_ing}]/dl'
                    try:
                        ing_el = driver.find_element(By.XPATH, ing_xpath)
                        ing_text = ing_el.text.strip()
                        ingredients.append(ing_text)
                        print(f"      - {ing_text}")
                        idx_ing += 1
                    except:
                        break
                if not ingredients:
                    print("      [재료 정보 없음]")
                
                # (6) 만드는 방법(설명) 수집
                print("    🍳 만드는 방법:")
                steps = []
                idx_step = 1
                while True:
                    step_xpath = f'//*[@id="contents"]/div/div[1]/div/div[3]/div/div/div/div[{idx_step}]/p'
                    try:
                        step_el = driver.find_element(By.XPATH, step_xpath)
                        step_text = step_el.text.strip()
                        steps.append(step_text)
                        print(f"      {idx_step}. {step_text}")
                        idx_step += 1
                    except:
                        break
                if not steps:
                    print("      [만드는 방법 없음]")
                
                # (7) 과정 이미지 수집
                print("    🖼 조리 이미지:")
                step_imgs = []
                idx_img = 1
                while True:
                    img_xpath = f'//*[@id="contents"]/div/div[1]/div/div[2]/div[3]/div/div/div[{idx_img}]/img'
                    try:
                        img_el = driver.find_element(By.XPATH, img_xpath)
                        img_src = img_el.get_attribute("src")
                        step_imgs.append(img_src)
                        print(f"      {idx_img}. {img_src}")
                        idx_img += 1
                    except:
                        break
                if not step_imgs:
                    print("      [조리 이미지 없음]")
                
                # 결과 저장
                data = {
                    "page": page,
                    "index": i,
                    "name": name,
                    "thumbnail_url": thumbnail_url,
                    "tags": tags,
                    "ingredients": ingredients,
                    "steps": steps,
                    "step_images": step_imgs
                }
                results.append(data)
                
                print(f"    ✅ [{page}페이지-{i}번째] '{name}' 수집 완료")
                
                # 뒤로가기
                driver.back()
                time.sleep(1)

            except Exception as e:
                print(f"    [오류] {page}페이지-{i}번째 접근 중 예외 발생: {e}")

finally:
    # 전체 완료
    print("\n=== 크롤링 완료 ===")
    print(f"총 {len(results)}건의 레시피 정보를 수집했습니다.")

    # 2) pandas에 저장
    df = pd.DataFrame(results)


    # 사용 후 드라이버 종료
    driver.quit()



=== [템플푸드] 페이지 1 ===
  - 1페이지, 1번째 레시피 확인 중...
    🥬 재료 목록:
      - 겉절이채소
40 g
      - 깨소금
약간
      - 당근 양념
3 테이블스푼
      - (당근 양념)
      - 당근
100 g
      - 매실청
2 테이블스푼
      - 올리브오일
1/3 컵
      - 소금
0.3 테이블스푼
    🍳 만드는 방법:
      [만드는 방법 오류 발생] Message: javascript error: Cannot read properties of undefined (reading 'slickNext')
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF610474C25+3179557]
	(No symbol) [0x00007FF6100D88A0]
	(No symbol) [0x00007FF60FF691CA]
	(No symbol) [0x00007FF60FF70B98]
	(No symbol) [0x00007FF60FF73F11]
	(No symbol) [0x00007FF610011652]
	(No symbol) [0x00007FF60FFE7C2A]
	(No symbol) [0x00007FF6100102F3]
	(No symbol) [0x00007FF60FFE7A03]
	(No symbol) [0x00007FF60FFB06D0]
	(No symbol) [0x00007FF60FFB1983]
	GetHandleVerifier [0x00007FF6104D67CD+3579853]
	GetHandleVerifier [0x00007FF6104ED1D2+3672530]
	GetHandleVerifier [0x00007FF6104E2153+3627347]
	GetHandleVerifier [0x00007FF61024092A+868650]
	(No symbol) [0x00007FF6100E2FFF]
	(No 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF610474C25+3179557]
	(No symbol) [0x00007FF6100D88A0]
	(No symbol) [0x00007FF60FF691CA]
	(No symbol) [0x00007FF60FF41753]
	(No symbol) [0x00007FF60FFEF83E]
	(No symbol) [0x00007FF61000FBE2]
	(No symbol) [0x00007FF60FFE7A03]
	(No symbol) [0x00007FF60FFB06D0]
	(No symbol) [0x00007FF60FFB1983]
	GetHandleVerifier [0x00007FF6104D67CD+3579853]
	GetHandleVerifier [0x00007FF6104ED1D2+3672530]
	GetHandleVerifier [0x00007FF6104E2153+3627347]
	GetHandleVerifier [0x00007FF61024092A+868650]
	(No symbol) [0x00007FF6100E2FFF]
	(No symbol) [0x00007FF6100DF4A4]
	(No symbol) [0x00007FF6100DF646]
	(No symbol) [0x00007FF6100CEAA9]
	BaseThreadInitThunk [0x00007FFEADBC259D+29]
	RtlUserThreadStart [0x00007FFEAF26AF38+40]


In [113]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

base_url = "https://www.koreatemplefood.com/templefood/recipe/index.html?page={}&plist=&find_field=title&find_word=&find_state=&find_ordby=&conf=&mode=&jijum_uid=&s_season22=&s_food22=%EA%B9%80%EC%B9%98%EB%A5%98"

results = []

try:
    # 1) 페이지 루프
    for page in range(1, 14):
        print(f"\n=== [템플푸드] 페이지 {page} ===")
        driver.get(base_url.format(page))
        time.sleep(1)
        
        # 각 페이지에 6개의 레시피 목록
        for i in range(1, 7):
            print(f"  - {page}페이지, {i}번째 레시피 확인 중...")
            try:
                # (1) 레시피 이름
                name_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[2]'
                name_element = wait.until(EC.presence_of_element_located((By.XPATH, name_xpath)))
                name = name_element.text.strip()
                
                # (2) 썸네일 (CSS background-image)
                thumb_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[1]/span'
                thumb_element = driver.find_element(By.XPATH, thumb_xpath)
                # background-image: url("...")
                thumb_style = thumb_element.value_of_css_property('background-image')
                if 'url("' in thumb_style:
                    thumbnail_url = thumb_style.split('url("')[1].split('")')[0]
                else:
                    thumbnail_url = ""
                
                # (3) 태그 목록 (여러 개)
                tag_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[1]/span'
                tag_elements = driver.find_elements(By.XPATH, tag_xpath)
                tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

                # (4) 상세페이지 진입
                list_item = driver.find_element(By.XPATH, f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a')
                driver.execute_script("arguments[0].click();", list_item)
                time.sleep(1)
                
                # (5) 재료 수집
                print("    🥬 재료 목록:")
                ingredients = []
                idx_ing = 1
                while True:
                    ing_xpath = f'//*[@id="contents"]/div/div[1]/div/ol/li[{idx_ing}]/dl'
                    try:
                        ing_el = driver.find_element(By.XPATH, ing_xpath)
                        ing_text = ing_el.text.strip()
                        ingredients.append(ing_text)
                        print(f"      - {ing_text}")
                        idx_ing += 1
                    except:
                        break
                if not ingredients:
                    print("      [재료 정보 없음]")
                
                # (6) 만드는 방법(설명) 수집 부분 수정
                print("    🍳 만드는 방법:")
                steps = []
                try:
                    # div[@class="recipe_how_slick"] 내부의 div[@class="item"] 안에 존재하는 p[@class="txt"] 전부 추출
                    step_xpath = '//div[contains(@class,"recipe_how_slick")]//div[@class="item"]/p[@class="txt"]'
                    step_els = driver.find_elements(By.XPATH, step_xpath)
                    
                    for idx, step_el in enumerate(step_els, start=1):
                        step_text = step_el.text.strip()
                        steps.append(step_text)
                        print(f"      {idx}. {step_text}")
                    
                    if not steps:
                        print("      [만드는 방법 없음]")
                except Exception as e:
                    print("      [만드는 방법 영역에서 오류 발생]", e)
                
                # (7) 과정 이미지 수집
                print("    🖼 조리 이미지:")
                step_imgs = []
                idx_img = 1
                while True:
                    # 기존 코드 방식 그대로 유지
                    img_xpath = f'//*[@id="contents"]/div/div[1]/div/div[2]/div[3]/div/div/div[{idx_img}]/img'
                    try:
                        img_el = driver.find_element(By.XPATH, img_xpath)
                        img_src = img_el.get_attribute("src")
                        step_imgs.append(img_src)
                        print(f"      {idx_img}. {img_src}")
                        idx_img += 1
                    except:
                        break
                if not step_imgs:
                    print("      [조리 이미지 없음]")
                
                # 결과 저장
                data = {
                    "page": page,
                    "index": i,
                    "name": name,
                    "thumbnail_url": thumbnail_url,
                    "tags": tags,
                    "ingredients": ingredients,
                    "steps": steps,
                    "step_images": step_imgs
                }
                results.append(data)
                
                print(f"    ✅ [{page}페이지-{i}번째] '{name}' 수집 완료")
                
                # 뒤로가기
                driver.back()
                time.sleep(1)

            except Exception as e:
                print(f"    [오류] {page}페이지-{i}번째 접근 중 예외 발생: {e}")

finally:
    # 전체 완료
    print("\n=== 크롤링 완료 ===")
    print(f"총 {len(results)}건의 레시피 정보를 수집했습니다.")

    # 2) pandas에 저장
    df = pd.DataFrame(results)
    # 예: df.to_excel("recipe_results.xlsx", index=False)

    # 사용 후 드라이버 종료
    driver.quit()



=== [템플푸드] 페이지 1 ===
  - 1페이지, 1번째 레시피 확인 중...
    🥬 재료 목록:
      - 겉절이채소
40 g
      - 깨소금
약간
      - 당근 양념
3 테이블스푼
      - (당근 양념)
      - 당근
100 g
      - 매실청
2 테이블스푼
      - 올리브오일
1/3 컵
      - 소금
0.3 테이블스푼
    🍳 만드는 방법:
      [만드는 방법 없음]
    🖼 조리 이미지:
      1. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11139_a.jpg
      2. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11139_b.jpg
    ✅ [1페이지-1번째] '당근 양념겉절이' 수집 완료
  - 1페이지, 2번째 레시피 확인 중...
    🥬 재료 목록:
      - 연근
100 g
      - 소금
적당량
      - 미나리
20 g
      - 배
1/6 개
      - 잣
1 테이블스푼
      - 다시마 국물
1 컵
    🍳 만드는 방법:
      [만드는 방법 없음]
    🖼 조리 이미지:
      1. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11111_a.jpg
      2. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11111_b.jpg
    ✅ [1페이지-2번째] '연근백김치' 수집 완료
  - 1페이지, 3번째 레시피 확인 중...
    🥬 재료 목록:
      - 겉절이채소
50 g
      - 버섯
50

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF610474C25+3179557]
	(No symbol) [0x00007FF6100D88A0]
	(No symbol) [0x00007FF60FF691CA]
	(No symbol) [0x00007FF60FF41753]
	(No symbol) [0x00007FF60FFEF83E]
	(No symbol) [0x00007FF61000FBE2]
	(No symbol) [0x00007FF60FFE7A03]
	(No symbol) [0x00007FF60FFB06D0]
	(No symbol) [0x00007FF60FFB1983]
	GetHandleVerifier [0x00007FF6104D67CD+3579853]
	GetHandleVerifier [0x00007FF6104ED1D2+3672530]
	GetHandleVerifier [0x00007FF6104E2153+3627347]
	GetHandleVerifier [0x00007FF61024092A+868650]
	(No symbol) [0x00007FF6100E2FFF]
	(No symbol) [0x00007FF6100DF4A4]
	(No symbol) [0x00007FF6100DF646]
	(No symbol) [0x00007FF6100CEAA9]
	BaseThreadInitThunk [0x00007FFEADBC259D+29]
	RtlUserThreadStart [0x00007FFEAF26AF38+40]


In [111]:
import requests
from bs4 import BeautifulSoup

def get_steps():
    # 1) 요청할 URL 지정
    url = "https://www.koreatemplefood.com/templefood/recipe/index.html?plist=&find_field=title&find_word=&find_state=&find_ordby=&conf=&mode=&jijum_uid=&s_season22=&s_food22=%EA%B9%80%EC%B9%98%EB%A5%98&bmain=view&uid=11139"

    # 2) 웹페이지 요청/응답
    response = requests.get(url)
    response.encoding = 'utf-8'  # 혹은 .apparent_encoding 사용 가능
    html = response.text

    # 3) BeautifulSoup 으로 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # 4) "만드는 방법" 영역의 p.txt 태그 추출
    steps = soup.select(".recipe_how_slick .item p.txt")

    # 5) 수집된 텍스트를 리스트 형태로 만들어 반환
    step_texts = [step.get_text(strip=True) for step in steps]
    return step_texts

if __name__ == "__main__":
    step_text_list = get_steps()
    for idx, step_text in enumerate(step_text_list, start=1):
        print(f"{idx}. {step_text}")


1. 1. 겉절이채소는 양상추, 치커리, 샐러드채소를 준비하여 물에 씻어 먹기 좋은 크기로 썬다.
2. 2. 당근 양념을 만든다.
당근은 껍질을 벗겨 믹서에 곱게 갈아 나머지 양념과 골고루 섞는다.
3. 3. 겉절이채소에 당근 양념을 넣어 골고루 무친다.


In [115]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

base_url = "https://www.koreatemplefood.com/templefood/recipe/index.html?page={}&plist=&find_field=title&find_word=&find_state=&find_ordby=&conf=&mode=&jijum_uid=&s_season22=&s_food22=%EA%B9%80%EC%B9%98%EB%A5%98"

results = []

try:
    for page in range(1, 14):
        print(f"\n=== [템플푸드] 페이지 {page} ===")
        driver.get(base_url.format(page))
        time.sleep(1)
        
        for i in range(1, 7):
            print(f"  - {page}페이지, {i}번째 레시피 확인 중...")
            try:
                # (1) 레시피 이름
                name_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[2]'
                name_element = wait.until(EC.presence_of_element_located((By.XPATH, name_xpath)))
                name = name_element.text.strip()
                
                # (2) 썸네일
                thumb_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[1]/span'
                thumb_element = driver.find_element(By.XPATH, thumb_xpath)
                thumb_style = thumb_element.value_of_css_property('background-image')
                if 'url("' in thumb_style:
                    thumbnail_url = thumb_style.split('url("')[1].split('")')[0]
                else:
                    thumbnail_url = ""
                
                # (3) 태그 목록
                tag_xpath = f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a/div[2]/p[1]/span'
                tag_elements = driver.find_elements(By.XPATH, tag_xpath)
                tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

                # (4) 상세페이지 진입
                list_item = driver.find_element(By.XPATH, f'//*[@id="contents"]/div/div[2]/ul/li[{i}]/a')
                driver.execute_script("arguments[0].click();", list_item)
                time.sleep(1)
                
                # (5) 재료 수집
                print("    🥬 재료 목록:")
                ingredients = []
                idx_ing = 1
                while True:
                    ing_xpath = f'//*[@id="contents"]/div/div[1]/div/ol/li[{idx_ing}]/dl'
                    try:
                        ing_el = driver.find_element(By.XPATH, ing_xpath)
                        ing_text = ing_el.text.strip()
                        ingredients.append(ing_text)
                        print(f"      - {ing_text}")
                        idx_ing += 1
                    except:
                        break
                if not ingredients:
                    print("      [재료 정보 없음]")
                
                # (6) "만드는 방법" 섹션: Selenium 대신 BeautifulSoup으로 파싱
                print("    🍳 만드는 방법:")
                steps = []
                
                # 6-1) 현재 페이지의 최종 HTML 소스를 가져온다
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                
                # 6-2) ".recipe_how_slick .item p.txt" 로 모두 찾는다
                step_tags = soup.select(".recipe_how_slick .item p.txt")
                
                for idx_s, tag in enumerate(step_tags, start=1):
                    step_text = tag.get_text(strip=True)
                    steps.append(step_text)
                    print(f"      {idx_s}. {step_text}")
                
                if not steps:
                    print("      [만드는 방법 없음]")
                
                # (7) 과정 이미지 수집 (기존 XPath 로직 유지)
                print("    🖼 조리 이미지:")
                step_imgs = []
                idx_img = 1
                while True:
                    img_xpath = f'//*[@id="contents"]/div/div[1]/div/div[2]/div[3]/div/div/div[{idx_img}]/img'
                    try:
                        img_el = driver.find_element(By.XPATH, img_xpath)
                        img_src = img_el.get_attribute("src")
                        step_imgs.append(img_src)
                        print(f"      {idx_img}. {img_src}")
                        idx_img += 1
                    except:
                        break
                if not step_imgs:
                    print("      [조리 이미지 없음]")
                
                # 결과 저장
                data = {
                    "page": page,
                    "index": i,
                    "name": name,
                    "thumbnail_url": thumbnail_url,
                    "tags": tags,
                    "ingredients": ingredients,
                    "steps": steps,
                    "step_images": step_imgs
                }
                results.append(data)
                
                print(f"    ✅ [{page}페이지-{i}번째] '{name}' 수집 완료")
                
                # 뒤로가기
                driver.back()
                time.sleep(1)

            except Exception as e:
                print(f"    [오류] {page}페이지-{i}번째 접근 중 예외 발생: {e}")

finally:
    print("\n=== 크롤링 완료 ===")
    print(f"총 {len(results)}건의 레시피 정보를 수집했습니다.")

    df = pd.DataFrame(results)
    # df.to_excel("recipe_results.xlsx", index=False)
    
    driver.quit()



=== [템플푸드] 페이지 1 ===
  - 1페이지, 1번째 레시피 확인 중...
    🥬 재료 목록:
      - 겉절이채소
40 g
      - 깨소금
약간
      - 당근 양념
3 테이블스푼
      - (당근 양념)
      - 당근
100 g
      - 매실청
2 테이블스푼
      - 올리브오일
1/3 컵
      - 소금
0.3 테이블스푼
    🍳 만드는 방법:
      1. 1. 겉절이채소는 양상추, 치커리, 샐러드채소를 준비하여 물에 씻어 먹기 좋은 크기로 썬다.
      2. 2. 당근 양념을 만든다.
당근은 껍질을 벗겨 믹서에 곱게 갈아 나머지 양념과 골고루 섞는다.
      3. 3. 겉절이채소에 당근 양념을 넣어 골고루 무친다.
    🖼 조리 이미지:
      1. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11139_a.jpg
      2. https://www.koreatemplefood.com/INC/thumb05.php?file=/upload/templefood/FoodDbRecipe11139_b.jpg
    ✅ [1페이지-1번째] '당근 양념겉절이' 수집 완료
  - 1페이지, 2번째 레시피 확인 중...
    🥬 재료 목록:
      - 연근
100 g
      - 소금
적당량
      - 미나리
20 g
      - 배
1/6 개
      - 잣
1 테이블스푼
      - 다시마 국물
1 컵
    🍳 만드는 방법:
      1. 1. 연근은 껍질을 벗기고 씻어서 얇게 썬다.
소금을 뿌려 10분 정도 절인다.
      2. 2. 미나리는 다듬어 2~3cm 길이로 썬다.
      3. 3. 배, 잣, 다시마 국물은 믹서에 넣고 갈아서 소금으로 간을 한다.
      4. 4. 절인 연근과 미나리에 국물을 붓고 익혀서 냉장 보관한다.
    🖼 조리 이미지:
     

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF610474C25+3179557]
	(No symbol) [0x00007FF6100D88A0]
	(No symbol) [0x00007FF60FF68FFC]
	(No symbol) [0x00007FF60FFAF8DF]
	(No symbol) [0x00007FF60FFE7AF2]
	(No symbol) [0x00007FF60FFE244E]
	(No symbol) [0x00007FF60FFE14F9]
	(No symbol) [0x00007FF60FF355E5]
	GetHandleVerifier [0x00007FF6104D67CD+3579853]
	GetHandleVerifier [0x00007FF6104ED1D2+3672530]
	GetHandleVerifier [0x00007FF6104E2153+3627347]
	GetHandleVerifier [0x00007FF61024092A+868650]
	(No symbol) [0x00007FF6100E2FFF]
	(No symbol) [0x00007FF60FF341FF]
	GetHandleVerifier [0x00007FF61055FA28+4141608]
	BaseThreadInitThunk [0x00007FFEADBC259D+29]
	RtlUserThreadStart [0x00007FFEAF26AF38+40]


In [117]:
df

,page,index,name,thumbnail_url,tags,ingredients,steps,step_images
0,1,1,당근 양념겉절이,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 여름, 부식, 김치류, 무치기]","[겉절이채소\n40 g, 깨소금\n약간, 당근 양념\n3 테이블스푼, (당근 양념)...","[1. 겉절이채소는 양상추, 치커리, 샐러드채소를 준비하여 물에 씻어 먹기 좋은 크...",[https://www.koreatemplefood.com/INC/thumb05.p...
1,1,2,연근백김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 가을, 부식, 김치류, 발효]","[연근\n100 g, 소금\n적당량, 미나리\n20 g, 배\n1/6 개, 잣\n1...",[1. 연근은 껍질을 벗기고 씻어서 얇게 썬다.\n소금을 뿌려 10분 정도 절인다....,[https://www.koreatemplefood.com/INC/thumb05.p...
2,1,3,늙은 호박 양념겉절이,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 겨울, 부식, 김치류, 무치기]","[겉절이채소\n50 g, 버섯\n50 g, 식용유\n적당량, 소금\n약간, (늙은 ...","[1. 겉절이채소는 다듬어 먹기 좋게 썬다., 2. 버섯은 양송이버섯, 느타리버섯,...",[https://www.koreatemplefood.com/INC/thumb05.p...
3,1,4,수삼물김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 겨울, 부식, 김치류, 발효]","[배추\n4 장, 무\n1/4 개, 수삼\n1 개, 미나리\n50 g, 청고추\n1...","[1. 배추는 깨끗이 씻어 3 × 3cm 크기로 자른다., 2. 무는 깨끗이 씻어 ...",[https://www.koreatemplefood.com/INC/thumb05.p...
4,1,5,고추김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 여름, 부식, 김치류, 발효]","[아삭이고추\n12 개, 무\n1/3 개, 오이\n1/2 개, 당근\n50 g, 배...","[1. 무는 깨끗이 씻어 껍질을 벗기고 5 × 0.3cm 크기로 채 썬다., 2. ...",[https://www.koreatemplefood.com/INC/thumb05.p...
...,...,...,...,...,...,...,...,...
64,11,5,동치미,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 겨울, 부식, 김치류, 발효]","[무\n2 개, 생강\n20 g, 고추씨\n1/2 컵, 청각\n50 g, 청고추\n...",[1. 1. 무는 단단하고 작은 것으로 골라 무청과 잔털을 제거한 뒤 깨끗이 씻어 ...,[https://www.koreatemplefood.com/INC/thumb05.p...
65,11,6,배추김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 겨울, 부식, 김치류, 발효]","[배추\n1 통, 고춧가루\n1/2 컵, 건고추\n10 개, 건표고\n5 개, 늙은...","[1. 1. 배추는 뿌리 밑동과 겉잎을 다듬고, 길이로 포기의 반만 칼집을 넣어 손...",[https://www.koreatemplefood.com/INC/thumb05.p...
66,12,1,연근김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 겨울, 부식, 김치류, 발효]","[연근\n1 개, 밤\n2 개, 미나리\n30 g, 홍고추\n3 개, (양념), 채...","[1. 1. 연근은 깨끗이 씻어 껍질을 벗겨 0.5cm 두께로 썬다., 2. 2. ...",[https://www.koreatemplefood.com/INC/thumb05.p...
67,12,2,콩잎김치,https://www.koreatemplefood.com/INC/thumb10.ph...,"[계절식, 여름, 부식, 김치류, 발효]","[콩잎\n200 g, 청고추\n4 개, 생강\n10 g, 보리쌀\n1/4 컵, 된장...",[1. 1. 콩잎은 너무 여리면 풋내가 나기 때문에 진녹색이 되기 직전의 것으로 골...,[https://www.koreatemplefood.com/INC/thumb05.p...


In [123]:
df['name'][0]

'당근 양념겉절이'

In [129]:
df['ingredients'][0]

['겉절이채소\n40 g',
 '깨소금\n약간',
 '당근 양념\n3 테이블스푼',
 '(당근 양념)',
 '당근\n100 g',
 '매실청\n2 테이블스푼',
 '올리브오일\n1/3 컵',
 '소금\n0.3 테이블스푼']

In [133]:
df['steps'][0]

['1. 겉절이채소는 양상추, 치커리, 샐러드채소를 준비하여 물에 씻어 먹기 좋은 크기로 썬다.',
 '2. 당근 양념을 만든다.\n당근은 껍질을 벗겨 믹서에 곱게 갈아 나머지 양념과 골고루 섞는다.',
 '3. 겉절이채소에 당근 양념을 넣어 골고루 무친다.']